# **Project: Apriori Algorithm for Finding Frequent Itemsets with PySpark**

## Task 1: Import the Libraries and Set Up the Environment

In [1]:
import itertools
import findspark
findspark.init()
import pyspark

In [5]:
import os
import findspark

# Set environment variables within the notebook
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home'  # Verify this path
os.environ['SPARK_HOME'] = '/opt/homebrew/opt/apache-spark'  # Verify this path
findspark.init()

from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("example").getOrCreate()



/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: Permission denied
/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: exec: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: cannot execute: Undefined error: 0


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [4]:
spark.getOrCreate()

/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: Permission denied
/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: exec: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: cannot execute: Undefined error: 0


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:



data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])

df.show()


In [10]:
conf = pyspark.SparkConf()
conf.setAppName('apriori')
conf.setMaster('local')
context = pyspark.SparkContext(conf=conf)


/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: Permission denied
/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: exec: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: cannot execute: Undefined error: 0


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [11]:
from pyspark import SparkContext
SparkContext.getOrCreate().stop()

/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: Permission denied
/opt/homebrew/opt/apache-spark/bin/load-spark-env.sh: line 2: exec: /opt/homebrew/Cellar/apache-spark/3.5.1/libexec/bin/load-spark-env.sh: cannot execute: Undefined error: 0


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:

session = pyspark.sql.SparkSession(context)

In [ ]:
conf.getAll()

# Task 2: Generate Combinations—Parent Intersection Property

In [ ]:
def pre_check(freq_k_1, k):
    k_size_comb = []
    for i in range(len(freq_k_1)):
        x = set(freq_k_1[i])
        for j in range(len(freq_k_1)):
            y = set(freq_k_1[j])
            if j<i:
                if len(x.intersection(y)) >= (k-2):
                    k_size_comb.append(tuple(sorted(list(x.union(y)))))
    return k_size_comb

## Task 3: Generate Combinations—Subset Frequency Property

In [ ]:
def post_check(k_size_comb, freq_k_1, k):
    filtered = []
    for  comb in  k_size_comb:
        flag = False
        for sub_comb in itertools.combinations(comb, k-1):
            if sub_comb not in freq_k_1:
                flag = True
        if flag == False:
            filtered.append(tuple(comb))
    return filtered

## Task 4: Count Check

In [ ]:
def count_check(filtered, lines, supCount):
    results = []
    counts = dict(zip(filtered, [0]*len(filtered)))
    for combination in filtered:
        present = [False]*len(combination)
        for i in range(len(combination)):
            for line in lines: 
                if combination[i] in line:
                    present[i] = True
                if all(present):
                    counts[combination] +=1

    for word, count in counts.items():
        if (count>=supCount):
            results.append(word)
    return results

## Task 5: Generate k-Size Combinations

In [ ]:
def generator(freq_k_1, k, partition, support):
    
    lines = list(partition)
    supCount = len(lines)*support

    k_size_comb = pre_check(freq_k_1, k)
    
    filtered = post_check(k_size_comb, freq_k_1, k)
    
    return count_check(filtered, lines, supCount)

## Task 6: Generate Singles

In [ ]:
def get_singles(lines, support):
    supCount = len(list(lines))*support
    vocab = set([])
    for line in lines:
        for word in line:
            vocab.add(word)
    counts = dict(zip(vocab, [0]*len(list(vocab))))
    combinations = []
    for line in lines:
        for word in line:
            counts[word] +=1
    for word, count in counts.items():
        if (count>=supCount):
            combinations.append(tuple((word,))) 
    return sorted(combinations)

## Task 7: The Worker Partition Mapper

In [ ]:
seq_len = context.broadcast(2)

In [ ]:
def apriori(iterator):
    partition = []
    for v in iterator:
        partition.append(v)
    support = sup.value
    results= get_singles(partition, support)
    print('starting with', results)

    for k in range(2, seq_len.value+1):
        print('sequence length', k)
     
        combos = generator(results, k, partition, support)

        if len(combos) == 0:
            print('ending at sequence length' ,k-1)
            return results

        results = combos
    return results

## Task 8: Load Data and Preprocess

In [ ]:
rdd = context.textFile("usercode/Dataset.csv")
tagsheader = rdd.first() 
tags = context.parallelize(tagsheader)
seq_len = context.broadcast(3)
data = rdd.subtract(tags)
length = context.broadcast(data.count())
sup = context.broadcast(0.03)
lines = data.map(lambda x: x.lstrip('"').rstrip('"').split(','))

## Task 9: The Distributed Transform 

In [ ]:
freq = lines.mapPartitions(apriori)
freq = freq.distinct()
comb = freq.collect()
print("Possible frequent itemset(s):\n", comb)

## Task 10: Auxiliary Function to Check Presence

In [ ]:
def auxiliary(row, combinations):
    present= []
    for combination in combinations:
        presence = [False]*len(combination)
        for i in range(len(combination)):
            presence[i] = combination[i] in row
        if all(presence):
            present+=[combination]
    return present

## Task 11: Count Check at Master

In [ ]:
comb = context.broadcast(comb)
freq1 = lines.map(lambda x: [(key, 1) for key in auxiliary(x, comb.value)]).filter(lambda x: len(x)>0)

freq2 = freq1.flatMap(lambda x: x)
freq3 = freq2.reduceByKey(lambda x, y: x+y)
freq4 = freq3.filter(lambda x: x[1]>sup.value*length.value).map(lambda x: x[0])
freq4.collect()

# End

In [4]:
import os

# Set JAVA_HOME and SPARK_HOME
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home'
os.environ['SPARK_HOME'] = '/opt/homebrew/opt/apache-spark' 
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['SPARK_HOME'] + '/bin:' + os.environ['PATH']
# Verify the environment variables are set
print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME'])
print(os.environ['PATH'])


/opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home
/opt/homebrew/opt/apache-spark
/opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home/bin:/opt/homebrew/opt/apache-spark/bin:/Users/nenadbozinovic/Documents/blog/venv_blog/bin:/Users/nenadbozinovic/.pyenv/shims:/Users/nenadbozinovic/.pyenv/bin:/Library/Frameworks/Python.framework/Versions/3.11/bin:/Library/Frameworks/Python.framework/Versions/3.12/bin:/Library/Frameworks/Python.framework/Versions/3.9/bin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Library/TeX/texbin:/Applications/quarto/bin


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example")


In [7]:
spark

In [8]:

data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])

df.show()

AttributeError: 'Builder' object has no attribute 'createDataFrame'